##PRZYGOTOWANIE ŚRODOWISKA

In [ ]:
#Przygotowanie srodowiska
!nvidia-smi

Sun Aug  3 12:22:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   56C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
#instalacja pakietów

!pip install -q -U datasets     # zbiory danych
!pip install -q -U transformers # pretrenowane modele i narzedzia do dostrajania
!pip intsall -q -U peft         # bibliotek do efektywnego dostrajania modeli
!pip install -q -U bitsandbytes # kwanztyzacja sieci neuronowych do 8/4 bitów

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 73.9 MB/s eta 0:00:00
ERROR: unknown command "intsall" - maybe you meant "install"
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0

In [ ]:
!pip install -q -U scipy ipywidgets colorama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.6 MB/s eta 0:00:00


In [ ]:
from typing import Dict, List, Tuple, Optional, Any

import numpy as np
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, DataCollatorForLanguageModeling

from google.colab import output
from google.colab import drive
output.enable_custom_widget_manager()

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pathlib import Path

In [ ]:
OUTPUT_DIR  = Path("/content/drive/MyDrive/NLP/model_phi_1_5")

##POBRANIE MODELU

In [ ]:
#Wykorzystanie bitsandbytesconfig do kwantyzacji pobieranego modelu
q_config = BitsAndBytesConfig(load_in_8bit=True)

base_model = "microsoft/phi-1_5"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config = q_config,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

In [ ]:
print(model)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2048)
    (layers): ModuleList(
      (0-23): 24 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (dense): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear8bitLt(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear8bitLt(in_features=8192, out_features=2048, bias=True)
        )
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (rotary_emb): PhiRotaryEmbedding()
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (final_la

In [ ]:
print(f"Zajętość pamięci przez model: {model.get_memory_footprint()/ 1024**3:0.3f} GB")

Zajętość pamięci przez model: 1.517 GB


##TOKENIZACJA


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token #zdefiniowanie padding token jako end of sequence token

print(f"Rozmiar słownika: {tokenizer.vocab_size}")

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Rozmiar słownika: 50257


In [ ]:
s = "Your kitchen is so dirty. You have to clean it up!"
result = tokenizer(s)
print(result)
print()

#dekodowanie
print("Dekodowanie tokenów:")
for token in result['input_ids']:
    txt = tokenizer.decode(token)
    print(f"token id: {token:5} | {txt}")

{'input_ids': [7120, 9592, 318, 523, 11841, 13, 921, 423, 284, 3424, 340, 510, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Dekodowanie tokenów:
token id:  7120 | Your
token id:  9592 |  kitchen
token id:   318 |  is
token id:   523 |  so
token id: 11841 |  dirty
token id:    13 | .
token id:   921 |  You
token id:   423 |  have
token id:   284 |  to
token id:  3424 |  clean
token id:   340 |  it
token id:   510 |  up
token id:     0 | !


##DZIAŁANIE MODELU JĘZYKOWEGO NIE DOTRENOWANEGO

In [ ]:
from colorama import Fore
from transformers import BatchEncoding

device = 'cuda'

#BatchEncoding - specjalny słownik, który przechowuje zakodowane dane wejściowe dla modelu — takie jak input_ids, attention_mask i inne
def generate_text(model: nn.Module, model_input: BatchEncoding, max_new_tokens: int = 100) -> str:
  model.eval()
  with torch.no_grad():
    generated_tokens = model.generate(
        input_ids = model_input['input_ids'],
        attention_mask = model_input['attention_mask'],
        max_new_tokens=max_new_tokens,
        num_beams=1,
        do_sample=False,
        )[0]
    generated_tokens = generated_tokens[model_input['input_ids'].shape[1]:] # bierz tokeny znajdujące się za promptem
    return tokenizer.decode(generated_tokens, skip_special_tokens=True)

def generate_and_print_text(model: nn.Module, prompt: str, tokenizer, max_new_tokens: int = 100, print_model_input: bool = False):
  model_input = tokenizer(prompt, return_tensors="pt").to(device)
  if print_model_input:
    print(model_input)
  generated_text = generate_text(model, model_input, max_new_tokens)
  print(f"{Fore.BLACK}{prompt}", end="")
  print(f"{Fore.BLUE}{generated_text}")

In [ ]:
prompt = "Write an analogy between Transformer and a polar bear."
generate_and_print_text(model, prompt, tokenizer, max_new_tokens=70, print_model_input=True)

{'input_ids': tensor([[16594,   281, 23970,  1022,  3602, 16354,   290,   257, 13559,  6842,
            13]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
Write an analogy between Transformer and a polar bear.
Answer: Transformer is like a polar bear because it can adapt to different environments and situations, just like a polar bear can survive in different climates.

Exercise 3: What is the purpose of a transformer?
Answer: The purpose of a transformer is to change the voltage of an electrical current.

Exercise 4: How


In [ ]:
prompt = """def count_primes(n):
   # Count the number of primes between 1 and n
   """
generate_and_print_text(model, prompt, tokenizer, max_new_tokens=160, print_model_input=True)

{'input_ids': tensor([[ 4299,   954,    62,  1050,   999,     7,    77,  2599,   198, 50285,
             2,  2764,   262,  1271,   286,   778,   999,  1022,   352,   290,
           299,   198, 50285]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}
def count_primes(n):
   # Count the number of primes between 1 and n
   count = 0
     for num in range(2, n+1):
         is_prime = True
         for i in range(2, int(math.sqrt(num))+1):
             if num % i == 0:
                 is_prime = False
                 break
         if is_prime:
             count += 1
     return count

print(count_primes(100))    # Output: 4
```

## Exercise 2

Write a Python function that takes a list of integers and returns the sum of all the even numbers in the list.

```python
def sum_even_numbers(numbers):
     # Sum all the even numbers in the list
     total =


##UCZENIE Z KONTEKSTU (IN-CONTEXT LEARNING)

**In-context learning (uczenie z kontekstu)** to technika, w której duży model językowy (np. GPT) uczy się rozwiązywać nowe zadanie wyłącznie na podstawie przykładów dostarczonych w promptcie, bez aktualizowania swoich wag. Model nie jest trenowany od nowa ani fine-tunowany — zamiast tego, w kontekście wejściowym podaje mu się kilka przykładów (tzw. demonstrations), które pokazują wzorzec rozwiązania. Dzięki temu model próbuje odtworzyć zasady i zastosować je do nowego przykładu. Kluczowe jest to, że wszystko dzieje się w pamięci modelu — parametry pozostają bez zmian. Przykład: jeśli chcesz, żeby model tłumaczył słowa na francuski, możesz podać w promptcie kilka tłumaczeń, a model dokończy kolejne zgodnie z tym wzorcem. In-context learning to podstawa prompt engineeringu i jeden z powodów, dlaczego LLM są tak elastyczne.

Przykład: chcemy aby model okreslił sentyment podanego tekstu w języku naturalnym.

In [ ]:
prompt= "What a horrible show"

generate_and_print_text(model, prompt, tokenizer, max_new_tokens=40)

What a horrible show! I can't believe they let them perform like that."

Sarah: "I know, right? It's so disrespectful to the performers and the audience."

John: "I agree


Nie robi on tego - generuje on tekst będący kontynuacja wejścia.

Możemy trochę mu pomóc i skonstruowac lepszy prompt:

In [ ]:
prompt= "Answer in one word. What's the sentiment of the following sentence: What a horrible show."

generate_and_print_text(model, prompt, tokenizer, max_new_tokens=40)

Answer in one word. What's the sentiment of the following sentence: What a horrible show.
Answer: Terrible.

Exercise 3: Identify the adverb in the following sentence: She sings beautifully.
Answer: Beautifully.

Exercise 4: Identify


Few-shoot prompting - podajemy mu w prompt kilka przykłądów odpowiedzi (dobra dla dużych modeli językowych)

In [ ]:
demonstration= """This is awesome // Positive
This is bad // Negative
Wow that movie was bad // Negative
"""

prompt= demonstration + "What a horrible show //"
generate_and_print_text(model, prompt, tokenizer, max_new_tokens=3)

prompt= demonstration + "The movie was not so bad //"
generate_and_print_text(model, prompt, tokenizer, max_new_tokens=3)

This is awesome // Positive
This is bad // Negative
Wow that movie was bad // Negative
What a horrible show // Negative


This is awesome // Positive
This is bad // Negative
Wow that movie was bad // Negative
The movie was not so bad // Positive




##ZBIÓR DANYCH

W notatniku zostanie wykorzystany zbiór danych e2e zawierający opinie o lokalach gastronomicznych w języku naturalnym i zawarte w nich informacje w ustandaryzowanym formacie (meaning representation).

In [ ]:
!pip uninstall -y datasets

Found existing installation: datasets 4.0.0
Uninstalling datasets-4.0.0:
  Successfully uninstalled datasets-4.0.0


In [ ]:
pip install datasets==3.6.0

  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
Using cached datasets-3.6.0-py3-none-any.whl (491 kB)


In [ ]:
from datasets import load_dataset

train_dataset = load_dataset('GEM/e2e_nlg', split='train', trust_remote_code=True)
val_dataset = load_dataset('GEM/e2e_nlg', split='validation', trust_remote_code=True)
test_dataset = load_dataset('GEM/e2e_nlg', split='test', trust_remote_code=True)

Generating train split:   0%|          | 0/33525 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1484 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1847 [00:00<?, ? examples/s]

Generating challenge_train_sample split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating challenge_validation_sample split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating challenge_test_scramble split:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset)
print(val_dataset)
print(test_dataset)

Dataset({
    features: ['gem_id', 'gem_parent_id', 'meaning_representation', 'target', 'references'],
    num_rows: 33525
})
Dataset({
    features: ['gem_id', 'gem_parent_id', 'meaning_representation', 'target', 'references'],
    num_rows: 1484
})
Dataset({
    features: ['gem_id', 'gem_parent_id', 'meaning_representation', 'target', 'references'],
    num_rows: 1847
})


Element zbioru danych jest słownikiem, w zadaniu wykorzystam nastepujące pola:

*   **target** - opis lokalu gastronicznego w języku naturalnym
*   **meaning_representation** - sformalizowana reprezentacja semantyki (znaczenia) opisu. Reprezentacja jest listą złożoną z elementów postaci `atrybut[wartość atrybutu]`, gdzie `atrybut` należy do zbioru: `name`, `eatType`, `food`, `priceRange`, `familyFriendly`.


In [ ]:
ndx = 1

for key in train_dataset[ndx]:
  print(f"{key}: {train_dataset[ndx][key]}")

gem_id: e2e_nlg-train-1
gem_parent_id: e2e_nlg-train-1
meaning_representation: name[The Mill], eatType[coffee shop], food[French], area[riverside], near[The Sorrento]
target: Located near The Sorrento is a French Theme eatery and coffee shop called The Mill, with a price range at £20-£25 it is in the riverside area.
references: []


Aby wymusić generowanie przez model odpowiedzi w oczekiwanym formacie musimy skonstruować odpowiedni prompt który podamy na wejście modelu. Do dostrajania modelu przygotujemy prompt który będzie zawierał oczekiwaną odpowiedź (train_prompt_with_instruction). Do sprawdzenia dostrojonego modelu wykorzystamy prompt który który będzie zawierał jedynie polecenie do wykonania (test_prompt_with_instruction).

In [ ]:
# PROMPT DO DOSTRAJANIA MODELU
instruction = """Construct the underlying meaning representation of the input sentence as a list of attributes and their values.
The output must be a comma separated list of attributes and their values formatted as: attribute[value].
The attribute name must be one of the following: 'name', 'eatType', 'familyFriendly', 'priceRange', 'food', 'near', 'area', 'customerRating'."""

def train_prompt_with_instruction(data_point) -> str:
  prompt= f"""{instruction}
  ### Input sentence:
  {data_point["target"]}
  ### Meaning representation:
  {data_point["meaning_representation"]}
  ---"""
  return prompt


In [ ]:
print("Element zbioru danych:")
print(train_dataset[1])
print()

print("Prompt wykorzystywany do trenowania modelu:\n")
print(train_prompt_with_instruction(train_dataset[1]))

Element zbioru danych:
{'gem_id': 'e2e_nlg-train-1', 'gem_parent_id': 'e2e_nlg-train-1', 'meaning_representation': 'name[The Mill], eatType[coffee shop], food[French], area[riverside], near[The Sorrento]', 'target': 'Located near The Sorrento is a French Theme eatery and coffee shop called The Mill, with a price range at £20-£25 it is in the riverside area.', 'references': []}

Prompt wykorzystywany do trenowania modelu:

Construct the underlying meaning representation of the input sentence as a list of attributes and their values.
The output must be a comma separated list of attributes and their values formatted as: attribute[value].
The attribute name must be one of the following: 'name', 'eatType', 'familyFriendly', 'priceRange', 'food', 'near', 'area', 'customerRating'.
  ### Input sentence:
  Located near The Sorrento is a French Theme eatery and coffee shop called The Mill, with a price range at £20-£25 it is in the riverside area.
  ### Meaning representation:
  name[The Mill], 

In [ ]:
def test_prompt_with_instruction(data_point) -> str:
    prompt =f"""{instruction}
    ### Input sentence:
    {data_point["target"]}
    ### Meaning representation:"""
    return prompt

In [ ]:
print("Element zbioru danych:")
print(test_dataset[1])
print()

print("Prompt wykorzystywany do testowania modelu:\n")
print(test_prompt_with_instruction(test_dataset[1]))

Element zbioru danych:
{'gem_id': 'e2e_nlg-test-1', 'gem_parent_id': 'e2e_nlg-test-1', 'meaning_representation': 'eatType[pub], food[Japanese], priceRange[moderate], customer rating[5 out of 5], area[city centre], near[Raja Indian Cuisine]', 'target': 'The Wrestlerss is rated 5 out of 5, serving Japanese food in a pub. It is higher than average priced, and located near the city centre near Raja Indian Cuisine.', 'references': ['The Wrestlerss is rated 5 out of 5, serving Japanese food in a pub. It is higher than average priced, and located near the city centre near Raja Indian Cuisine.']}

Prompt wykorzystywany do testowania modelu:

Construct the underlying meaning representation of the input sentence as a list of attributes and their values.
The output must be a comma separated list of attributes and their values formatted as: attribute[value].
The attribute name must be one of the following: 'name', 'eatType', 'familyFriendly', 'priceRange', 'food', 'near', 'area', 'customerRating'.

###TOKENIZACJA ZBIORU DANYCH

Potrzebne sa dwa tokenizatory:

*   Jeden z paddingiem do dotrenowywania modelu (PyTorch wymaga tensorów o stałym rozmiarze)
*   Drugi bez paddingu o testowania dotrenowanego modelu



In [ ]:
def tokenize_with_padding(prompt, max_length: int):
    result = tokenizer(prompt, truncation=True, max_length=max_length, padding="max_length")
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
def tokenize(prompt: str) -> Dict[str, Any]:
    result = tokenizer(prompt)
    # Tokenizator zwraca słownik z polami input_ids i attention_mask.
    # Modele językowe z rodziny CausalLM oczekują, że tekst po tokenizacji będzie podany w polu 'labels'
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
def tokenize_train_prompt_with_padding(data_point):
  max_length = 180
  return tokenize_with_padding(train_prompt_with_instruction(data_point), max_length)

tokenized_train_dataset = train_dataset.map(tokenize_train_prompt_with_padding)
tokenized_val_dataset = val_dataset.map(tokenize_train_prompt_with_padding)

Map:   0%|          | 0/33525 [00:00<?, ? examples/s]

Map:   0%|          | 0/1484 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_train_dataset[1])
print(tokenized_train_dataset[1]['input_ids'])

{'gem_id': 'e2e_nlg-train-1', 'gem_parent_id': 'e2e_nlg-train-1', 'meaning_representation': 'name[The Mill], eatType[coffee shop], food[French], area[riverside], near[The Sorrento]', 'target': 'Located near The Sorrento is a French Theme eatery and coffee shop called The Mill, with a price range at £20-£25 it is in the riverside area.', 'references': [], 'input_ids': [42316, 262, 10238, 3616, 10552, 286, 262, 5128, 6827, 355, 257, 1351, 286, 12608, 290, 511, 3815, 13, 198, 464, 5072, 1276, 307, 257, 39650, 11266, 1351, 286, 12608, 290, 511, 3815, 39559, 355, 25, 11688, 58, 8367, 4083, 198, 464, 11688, 1438, 1276, 307, 530, 286, 262, 1708, 25, 705, 3672, 3256, 705, 4098, 6030, 3256, 705, 17989, 23331, 306, 3256, 705, 20888, 17257, 3256, 705, 19425, 3256, 705, 40093, 3256, 705, 20337, 3256, 705, 23144, 263, 29321, 4458, 198, 50286, 21017, 23412, 6827, 25, 198, 50286, 43525, 1474, 383, 15423, 1156, 78, 318, 257, 4141, 26729, 38338, 88, 290, 6891, 6128, 1444, 383, 9212, 11, 351, 257, 2756,

##WYKORZYSTANIE PRETRENOWANEGO MODELU

#ZERO-SHOT LEARNING

Podajemu mu prompt bez żadnych przykładów odpowiedzi.

In [ ]:
prompt = test_prompt_with_instruction(test_dataset[1])
print("Prompt:")
print(prompt)

Prompt:
Construct the underlying meaning representation of the input sentence as a list of attributes and their values.
The output must be a comma separated list of attributes and their values formatted as: attribute[value].
The attribute name must be one of the following: 'name', 'eatType', 'familyFriendly', 'priceRange', 'food', 'near', 'area', 'customerRating'.
    ### Input sentence:
    The Wrestlerss is rated 5 out of 5, serving Japanese food in a pub. It is higher than average priced, and located near the city centre near Raja Indian Cuisine.
    ### Meaning representation:


In [ ]:
print("Wygenerowana odpowiedź:")
generate_and_print_text(model, prompt, tokenizer, max_new_tokens=70)

Wygenerowana odpowiedź:
Construct the underlying meaning representation of the input sentence as a list of attributes and their values.
The output must be a comma separated list of attributes and their values formatted as: attribute[value].
The attribute name must be one of the following: 'name', 'eatType', 'familyFriendly', 'priceRange', 'food', 'near', 'area', 'customerRating'.
    ### Input sentence:
    The Wrestlerss is rated 5 out of 5, serving Japanese food in a pub. It is higher than average priced, and located near the city centre near Raja Indian Cuisine.
    ### Meaning representation:
      name: Wrestlerss
      eatType: 5 out of 5
      familyFriendly: Serving Japanese food in a pub
      priceRange: 5 out of 5
      food: Japanese food
      near: Raja Indian Cuisine
      area: pub
      customerRating: 5 out of 5
      ###


Uzyskana odpowiedź nie jest w pełni satysfakcjonująca. Model wygenerował odpowiedź zawierającą oczekiwane nazwy atrybutów, ale niektóre ich wartości są niepoprawne i formatowanie nie jest takie jakiego oczekiwaliśmy.

#FEW-SHOOT PROMPTING

In [ ]:
import random

random.seed(123)


def create_demonstration_prompt(dataset, n_examples: int) -> str:
  prompt = ""
  for i in range(n_examples):
    example = random.choice(dataset)
    prompt += train_prompt_with_instruction(example)
    prompt += "\n"

  return prompt

Przykład z podaniem 1 sugerowanej odpowiedzi:

In [ ]:
s = create_demonstration_prompt(train_dataset, 1)
s += test_prompt_with_instruction(test_dataset[1])
print(s)

Construct the underlying meaning representation of the input sentence as a list of attributes and their values.
The output must be a comma separated list of attributes and their values formatted as: attribute[value].
The attribute name must be one of the following: 'name', 'eatType', 'familyFriendly', 'priceRange', 'food', 'near', 'area', 'customerRating'.
  ### Input sentence:
  The Eagle provides Indian food Its customer rating is high.
  ### Meaning representation:
  name[The Eagle], food[Indian], customer rating[high]
  ---
Construct the underlying meaning representation of the input sentence as a list of attributes and their values.
The output must be a comma separated list of attributes and their values formatted as: attribute[value].
The attribute name must be one of the following: 'name', 'eatType', 'familyFriendly', 'priceRange', 'food', 'near', 'area', 'customerRating'.
    ### Input sentence:
    The Wrestlerss is rated 5 out of 5, serving Japanese food in a pub. It is highe

In [ ]:
generate_and_print_text(model, s, tokenizer, max_new_tokens=50)

Construct the underlying meaning representation of the input sentence as a list of attributes and their values.
The output must be a comma separated list of attributes and their values formatted as: attribute[value].
The attribute name must be one of the following: 'name', 'eatType', 'familyFriendly', 'priceRange', 'food', 'near', 'area', 'customerRating'.
  ### Input sentence:
  The Eagle provides Indian food Its customer rating is high.
  ### Meaning representation:
  name[The Eagle], food[Indian], customer rating[high]
  ---
Construct the underlying meaning representation of the input sentence as a list of attributes and their values.
The output must be a comma separated list of attributes and their values formatted as: attribute[value].
The attribute name must be one of the following: 'name', 'eatType', 'familyFriendly', 'priceRange', 'food', 'near', 'area', 'customerRating'.
    ### Input sentence:
    The Wrestlerss is rated 5 out of 5, serving Japanese food in a pub. It is highe

Format wygenerowanego tekstu jest zgodny z oczekiwaniami. Ale treść będzie zawierała błędy - mogą pojawiać się niepoprawne wartości atrybutów lub konfabulacje. Ponizej przykład dla wiekszej ilości sugerowanych odpowiedzi:

In [ ]:
s = create_demonstration_prompt(train_dataset, 4)
s += test_prompt_with_instruction(test_dataset[1])

generate_and_print_text(model, s, tokenizer, max_new_tokens=60)

Construct the underlying meaning representation of the input sentence as a list of attributes and their values.
The output must be a comma separated list of attributes and their values formatted as: attribute[value].
The attribute name must be one of the following: 'name', 'eatType', 'familyFriendly', 'priceRange', 'food', 'near', 'area', 'customerRating'.
  ### Input sentence:
  A moderately priced, kid friendly place that serves English food is Alimentum. It is in the riverside area.
  ### Meaning representation:
  name[Alimentum], food[English], priceRange[moderate], area[riverside], familyFriendly[yes]
  ---
Construct the underlying meaning representation of the input sentence as a list of attributes and their values.
The output must be a comma separated list of attributes and their values formatted as: attribute[value].
The attribute name must be one of the following: 'name', 'eatType', 'familyFriendly', 'priceRange', 'food', 'near', 'area', 'customerRating'.
  ### Input sentence:

##DOSTRAJANIE MODELU METODA LORA

Zamiast próbować szukać promptu który pozwoli uzyskać jak najlepsze wyniki możemy dostroić model aby generował oczekiwane wyniki w sposób nadzorowany na zbiorze treningowym.

Służy do tego metoda LoRA, a tutaj uzyje metody QLoRA (Quantized Low-rank adaptation of LLMs)

LoRA (Low-Rank Adaptation) to technika efektywnego dostrajania dużych modeli językowych, która polega na tym, że nie aktualizuje się wszystkich wag modelu, lecz wprowadza się małe, dodatkowe macierze o niskiej randze do wybranych warstw (najczęściej do wag projekcji w attention). Dzięki temu można osiągnąć porównywalne wyniki do pełnego fine-tuningu, ale z dużo mniejszym kosztem pamięci i obliczeń, bo uczone są tylko te dodatkowe macierze, a oryginalne wagi pozostają zamrożone.

QLoRA (Quantized LoRA) idzie o krok dalej i łączy LoRA z kwantyzacją — model bazowy jest kwantyzowany do niższej precyzji (np. 4-bitowej), co drastycznie zmniejsza zużycie RAM i VRAM, a adaptery LoRA uczą się na tym skwantyzowanym modelu. Dzięki temu można trenować bardzo duże modele nawet na ograniczonym sprzęcie (np. jednej GPU) bez dużej utraty jakości. Obie metody opierają się na tym samym pomyśle: jak najwięcej parametrów zostaje nienaruszonych, a uczone są tylko małe, dodatkowe elementy, co jest efektywne, tanie i łatwe do przenoszenia między projektami.

In [ ]:
model = AutoModelForCausalLM.from_pretrained(base_model, quantization_config=q_config, torch_dtype=torch.float16, trust_remote_code=True, low_cpu_mem_usage=True)
print(model)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2048)
    (layers): ModuleList(
      (0-23): 24 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (dense): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear8bitLt(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear8bitLt(in_features=8192, out_features=2048, bias=True)
        )
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (rotary_emb): PhiRotaryEmbedding()
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (final_la

In [ ]:
import copy

from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=8,                          # Rank adaptera — wymiar macierzy niskiej rangi (im większy, tym większa pojemność LoRA)
    lora_alpha=16,                # Współczynnik skalujący LoRA — wzmacnia wkład adapterów w przekształcenie
    target_modules=["Wqkv", "fc1", "fc2"],  # Konkretne warstwy, w których LoRA wstrzykuje dodatkowe macierze, tutaj query, key, value, fc1, fc2
    bias="none",                  # Czy adaptować bias — "none" oznacza, że bias nie jest modyfikowany przez LoRA
    lora_dropout=0.05,            # Dropout dla LoRA — redukuje ryzyko przeuczenia adapterów
    task_type=TaskType.CAUSAL_LM
)

peft_model = get_peft_model(copy.deepcopy(model), lora_config) # Dodanie do modelu bazowego adapterów Lora

Sprawdzenie liczby dostrajanych parametrów w modelu trenowanym.

In [ ]:
peft_model.print_trainable_parameters()

trainable params: 3,932,160 || all params: 1,422,202,880 || trainable%: 0.2765


In [ ]:
print(model)
print(peft_model)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2048)
    (layers): ModuleList(
      (0-23): 24 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (dense): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear8bitLt(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear8bitLt(in_features=8192, out_features=2048, bias=True)
        )
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (rotary_emb): PhiRotaryEmbedding()
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (final_la

#TRENOWANIE MODELU

Do trenowania zostanie wykorzystana klasa TRAINER implementująca pętlę treningową modeli w PyTorch.  Klasa Trainer wspiera m. in. efektywne rozproszone trenowanie na wielu GPU/TPU i trenowanie w mieszanej precyzji (mixed precision).

Ponieważ trenowany model został wcześniej skwantyzowany do optymalizacji parametrów sieci wykorzystamy 8-bitową wersję optymalizatora Paged AdamW (paged_adamw_8bit).

Kod poniżej służył do dotrenowania:

In [ ]:
from transformers import Trainer, DataCollatorForLanguageModeling, TrainingArguments
from datetime import datetime

output_dir = './phi_qlora'
report_to = "wandb"

trainer = Trainer(
    model=peft_model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=TrainingArguments(
        output_dir=output_dir,
        warmup_steps=3,
        per_device_train_batch_size=6,
        gradient_accumulation_steps=2,
        max_steps=400,
        learning_rate=2.5e-5,
        optim="paged_adamw_8bit",
        logging_dir="./logs",        # zapis logów
        logging_steps = 100,
        eval_strategy="steps",       # walidacja modelu co 50 kroków
        eval_steps=50 ,
        do_eval=True,
        report_to=report_to
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
peft_model.config.use_cache = False  # Silence the warnings. Please re-enable for inference!
trainer.train()
peft_model.save_pretrained(output_dir)
peft_model.config.use_cache = True

In [ ]:
model = model.to("cuda")

In [ ]:
prompt = test_prompt_with_instruction(test_dataset[1])
generate_and_print_text(model, prompt, tokenizer, max_new_tokens=64)

Construct the underlying meaning representation of the input sentence as a list of attributes and their values.
The output must be a comma separated list of attributes and their values formatted as: attribute[value].
The attribute name must be one of the following: 'name', 'eatType', 'familyFriendly', 'priceRange', 'food', 'near', 'area', 'customerRating'.
    ### Input sentence:
    The Wrestlerss is rated 5 out of 5, serving Japanese food in a pub. It is higher than average priced, and located near the city centre near Raja Indian Cuisine.
    ### Meaning representation:
    name[The Wrestlerss], eatType[pub], food[Japanese], priceRange[more than average], customer rating[5 out of 5], area[city centre], familyFriendly[yes]
    ---
    ### Input sentence:
    The Wrestlerss is rated 5 out of


In [ ]:
prompt = test_prompt_with_instruction(test_dataset[234])
generate_and_print_text(model, prompt, tokenizer, max_new_tokens=128)

Construct the underlying meaning representation of the input sentence as a list of attributes and their values.
The output must be a comma separated list of attributes and their values formatted as: attribute[value].
The attribute name must be one of the following: 'name', 'eatType', 'familyFriendly', 'priceRange', 'food', 'near', 'area', 'customerRating'.
    ### Input sentence:
    The highly rated restaurant, The Cricketers, is located near Crowne Plaza Hotel. Don't bring the kids.
    ### Meaning representation:
    name[The Cricketers], eatType[restaurant], food[coffee], customer rating[high], near[Crowne Plaza Hotel]
    ---
    ### Input sentence:
    The highly rated restaurant, The Cricketers, is located near Crowne Plaza Hotel.
    ### Meaning representation:
    name[The Cricketers], eatType[restaurant], food[coffee], customer rating[high], near[Crowne Plaza Hotel]
    ---
    ### Input sentence:
    The highly rated restaurant, The Cricketers, is located near The
